In [1]:
old_business_codes = [2123]
old_business_code_with_new_code = {
#     1951: 9000,
#     1614: 9001,
#     2148: 9002,
#     2197: 9003,
#     2275: 9004,
#     2298: 9005,
#     2300: 9006,
#     2304: 9007,
#     2305: 9008,
#     2313: 9009,
#     2341: 9010,
#     2348: 9011,
#     2353: 9012,
#     2356: 9013,
#     2324: 9014,
#     2386: 9015,
#     2413: 9016,
#     2421: 9017,
#     2568: 9018,
#     2571: 9019,
#     2599: 9020,
#     2626: 9021,
#     2843: 9022,
#     2844: 9023
    2123: 9025
}


In [4]:
find_new_business_code(2123)

9025

In [3]:
def find_new_business_code(old_code):
    new_code = old_business_code_with_new_code[old_code]
    return new_code

In [5]:
import pandas as pd
import datetime
business_agent_obj = BusinessAgentMap.objects.filter(business__code__in=old_business_codes)
business_agent_list = business_agent_obj.values_list('id','business_id', 'business__code','agent_id', 'agent__user_profile_id', 'agent__user_profile__user_id', 'nominee_id')
business_agent_column = ['business_agent_map_id','business_id', 'business_code','agent_id', 'user_profile_id', 'user_id', 'nominee_id']
business_agent_df = pd.DataFrame(business_agent_list, columns=business_agent_column)
business_agent_df['new_business_code'] = business_agent_df.apply(lambda x: find_new_business_code(int(x['business_code'])), axis=1)

route_business_obj = RouteBusinessMap.objects.filter(business__code__in=old_business_codes)
route_business_list = list(route_business_obj.values_list('business_id', 'route_id'))
route_business_column = ['business_id', 'route_id']
route_business_df = pd.DataFrame(route_business_list, columns=route_business_column)
route_business_df = route_business_df.groupby('business_id')['route_id'].apply(list)
# merge route with business

final_merge_with_route_df = business_agent_df.merge(route_business_df, how='left', left_on='business_id', right_on='business_id')
final_merge_with_route_df

,business_agent_map_id,business_id,business_code,agent_id,user_profile_id,user_id,nominee_id,new_business_code,route_id
0,1950,1951,2123,1950,1962,1969,1950,9025,"[114, 115]"


In [13]:
for index, row  in final_merge_with_route_df.iterrows():
    # dupilicate    
    user_obj = User.objects.filter(id=row['user_id']).values()[0]
    user_obj['id'] = None
    user_obj['username'] = row['new_business_code']
    new_user_obj = User.objects.create(**user_obj)
    print('User created for - {}'.format(user_obj['username']))
    
#   Dupilcate user profile and map with created user
    user_profile_obj = UserProfile.objects.filter(id=row['user_profile_id']).values()[0]
    user_profile_obj['id'] = None
    user_profile_obj['user_id'] = new_user_obj.id
    new_userprofile_obj = UserProfile.objects.create(**user_profile_obj)
    print('UserProfile created created')
    
#     Dupilcate the agent old as new agent
    agent_obj = Agent.objects.filter(id=row['agent_id']).values()[0]
    agent_obj['id'] = None
    agent_obj['user_profile_id'] = new_userprofile_obj.id
    new_agent_code = str(agent_obj['agent_code']) + '_WS'
    agent_obj['agent_code'] = new_agent_code
    new_agent_obj = Agent.objects.create(**agent_obj)
    print('Agent created with code of - {}'.format(agent_obj['agent_code']))
    
#     Dupicate business old as new business and change business_type to wholesale
    business_obj = Business.objects.filter(id=row['business_id']).values()[0]
    business_obj['id'] = None
    business_obj['code'] = row['new_business_code']
    business_obj['name'] = str(row['new_business_code']) + ' - Whole sale'
    business_obj['business_type_id'] = 9
    new_business_obj = Business.objects.create(**business_obj)
    print('Business created')
    
#     Dupicate the old agent bank details
    agent_bank_details_obj = AgentBankDetail.objects.filter(agent_id=row['agent_id']).values()[0]
    agent_bank_details_obj['id'] = None
    agent_bank_details_obj['agent_id'] = new_agent_obj.id
    new_agent_bank_details_obj = AgentBankDetail.objects.create(**agent_bank_details_obj)
    print('Agent bank details created and mapped with new agent')
    
#     Dupilcate nominee of the old agent 
    nominee_obj =  Nominee.objects.filter(id=row['nominee_id']).values()[0]
    nominee_obj['id'] = None
    new_nominee_obj = Nominee.objects.create(**nominee_obj)
    print('Nominee created for new agent')

#     Dupilicate agent wallet table
    agent_wallet_obj = AgentWallet.objects.filter(agent_id=row['agent_id']).values()[0]
    agent_wallet_obj['id'] = None
    agent_wallet_obj['agent_id'] = new_agent_obj.id
    new_agent_wallet_obj = AgentWallet.objects.create(**agent_wallet_obj)
    print('Agent Walet created and mapped with new agent')

#     Map business with agent and nominess
    business_agent_map_obj = BusinessAgentMap.objects.filter(id=row['business_agent_map_id']).values()[0]
    business_agent_map_obj['id'] = None
    business_agent_map_obj['agent_id'] = new_agent_obj.id
    business_agent_map_obj['business_id'] = new_business_obj.id
    business_agent_map_obj['nominee_id'] = new_nominee_obj.id
    new_business_agent_map_obj = BusinessAgentMap.objects.create(**business_agent_map_obj)
    print('Agent mapped with business and nominee')

#  Map new business with route
    for route_id in row['route_id']:
        RouteBusinessMap.objects.create(route_id=route_id, business_id=new_business_obj.id)
        print('Business map with route id of - {}'.format(route_id))
    print('------------------------------')

User created for - 9025
UserProfile created created
Agent created with code of - M646562_WS
Business created
Agent bank details created and mapped with new agent
Nominee created for new agent
Agent Walet created and mapped with new agent
Agent mapped with business and nominee
Business map with route id of - 114
Business map with route id of - 115
------------------------------


In [13]:
Agent.objects.get(user_profile__user__username=9021).agent_code

'2596_WS'

In [8]:
BusinessAgentMap.objects.get(business__code=9025).agent.user_profile.user.id

11165

In [14]:
Business.objects.get(code=9025).name

'9025Whole sale'

In [ ]:
Bu